In [1]:
import pandas as pd
import numpy as np
import glob
import functools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
RAW_FILE = 'NHANES-varnames_raw.xlsx'
NO_FLAG_FILE = 'NHANES-varnames_noflag.csv'
YES_FLAG_FILE = 'NHANES-varnames_yesflag.csv'
CLEAN_FILE = 'NHANES-clean.csv'
MISSING_FILE = 'NHANES-missing.csv'

COMPONENTS = ['Laboratory', 'Demographics', 'Questionnaire', 'Dietary', 'Examination']

YEARS = list(range(2003, 2016, 2))

In [3]:
# read raw file and append sheets 
no_flag_df = pd.DataFrame()
for c in COMPONENTS: 
    df = pd.read_excel(RAW_FILE, sheet_name = c)
    no_flag_df = no_flag_df.append(df)
    
# create year flags 
def var_in_year(row, year): 
    if (row['Begin Year'] <= year) & (row['EndYear'] >= year): 
        return 1
    return 0 

for y in YEARS: 
    v = 'flag_' + str(y)
    no_flag_df[v] = no_flag_df.apply(var_in_year, year = y, axis = 1)
    
flags = [col for col in no_flag_df if col.startswith('flag_')]
no_flag_df = (no_flag_df
              .groupby(['Variable Name', 
                        'Variable Description', 
                        'Component', 
                        'Data File Description'])[flags]
              .sum(axis = 1)
              .reset_index())

for y in YEARS: 
    v = 'flag_' + str(y)
    no_flag_df[v] = np.where(no_flag_df[v] == 0, 0, 1)

# write to csv
no_flag_df.to_csv(NO_FLAG_FILE, index = False)

In [4]:
# get list of variables to keep 
yes_flag_df = pd.read_csv(YES_FLAG_FILE)
keep_vars = (yes_flag_df[yes_flag_df['keep'] == 1]['Variable Name']
             .unique()
             .tolist())
keep_vars = set(keep_vars)

# loop over all years and components 
full_df_list = []
for y in YEARS:
    print(y)
    year_df_list = []
    for c in COMPONENTS: 
        print(c)
        path = 'csv_data/' + str(y) + '-' + str(y+1) + '/' + c + '/*.csv'
        for f in glob.iglob(path):
            df = pd.read_csv(f)
            df = df[df.columns[df.columns.isin(keep_vars)]]
            if (df.shape[1] > 1) & ('SEQN' in df.columns): 
                year_df_list.append(df)
                
    # merge (wide) within each year
    year_df = (functools.reduce(
        lambda df1, df2: pd.merge(df1, df2, on = 'SEQN', how = 'outer'), year_df_list)
               .drop_duplicates(subset = 'SEQN'))
    year_df['year'] = str(y) + '-' + str(y+1)
    year_df = year_df.loc[:,~year_df.columns.duplicated()]
    full_df_list.append(year_df)

# append (long) across years 
clean_df = pd.concat(full_df_list, axis = 0, sort = True)

# combine DRSTZ variables 
clean_df['DR1DRSTZ'] = np.where(clean_df['DR1DRSTZ_x'].notna(), 
                                clean_df['DR1DRSTZ_x'], clean_df['DR1DRSTZ_y'])
clean_df['DR2DRSTZ'] = np.where(clean_df['DR2DRSTZ_x'].notna(), 
                                clean_df['DR2DRSTZ_x'], clean_df['DR2DRSTZ_y'])
clean_df = clean_df.drop(columns = ['DR1DRSTZ_x', 'DR1DRSTZ_y', 
                                    'DR2DRSTZ_x', 'DR2DRSTZ_y'])

# combine day 1 and day 2 variables - based on DR1 and DR2 prefixes 
exclude_cols = ['year', 'SEQN', 
                'DR1DRSTZ','DR2DRSTZ', 
                'DR1_300', 'DR2_300', 
                'RIDAGEYR', 'RIAGENDR', 'INDFMPIR', 'RIDRETH1']

feature_cols = set()
for i in clean_df.columns: 
    if i not in exclude_cols: 
        feature_cols.add(i[3:])
    
for i in feature_cols: 
    clean_df[i] = clean_df['DR1' + i] + clean_df['DR2' + i]
    clean_df = clean_df.drop(columns = ['DR1' + i, 
                                        'DR2' + i])
    
# format and output to csv 
clean_df = round(clean_df, 5) 
clean_df.to_csv(CLEAN_FILE, index = False)

2003
Laboratory
Demographics
Questionnaire
Dietary
Examination
2005
Laboratory
Demographics
Questionnaire
Dietary
Examination
2007
Laboratory
Demographics
Questionnaire
Dietary
Examination
2009
Laboratory
Demographics
Questionnaire
Dietary
Examination
2011
Laboratory
Demographics
Questionnaire
Dietary
Examination
2013
Laboratory
Demographics
Questionnaire
Dietary
Examination
2015
Laboratory
Demographics
Questionnaire
Dietary
Examination
